In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from random import seed
from random import randrange

seed(10)

In [ ]:
class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

In [ ]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        
    # YOUR CODE HERE
    @staticmethod
    def entropy(x):
      # Count the number of each binary (0 and 1)
      # Convert to integers to avoid runtime errors
      counts = np.bincount(np.array(x, dtype=np.int64))
      # Probabilities of each class label
      ps = counts / len(x)

      # Caclulate entropy
      entropy = 0
      entropy = np.sum([p * np.log2(p) for p in ps if p > 0])

      return -entropy
      
    def __information_gain(self, parent, left_child, right_child):
      # Percentage of left child length over parent
      left_ratio = len(left_child) / len(parent)
      # Percentage of right child length over parent
      right_ratio = len(right_child) / len(parent)
      
      # One-liner which implements the previously discussed formula
      return self.entropy(parent) - (left_ratio * self.entropy(left_child) + right_ratio * self.entropy(right_child))
    
    def __best_split(self, X, y):
        '''
        Helper function, calculates the best split for given features and target
        
        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        # For every dataset feature
        for f_idx in range(n_cols):
            X_col = X[:, f_idx]

            # For every unique value of that feature
            for threshold in np.unique(X_col):
                # Construct a dataset and split it to the left and right parts
                # Left part includes records lower or equal to the threshold
                # Right part includes records higher than the threshold
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                # Do the calculation only if there's data in both subsets
                if len(df_left) > 0 and len(df_right) > 0:
                    # Obtain the value of the target variable for subsets
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    # Caclulate the information gain and save the split parameters
                    # if the current split if better then the previous best
                    gain = self.__information_gain(y, y_left, y_right)

                    if gain > best_info_gain:
                        best_info_gain = gain
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }

        return best_split

    def __build(self, X, y, depth=0):
        '''
        Helper recursive function, used to build a decision tree from the input data.
        
        :param X: np.array, features
        :param y: np.array or list, target
        :param depth: current depth of a tree, used as a stopping criteria
        :return: Node
        '''
        n_rows, n_cols = X.shape
        
        # Check to see if a node should be leaf node
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            # Get the best split
            best = self.__best_split(X, y)
            # If the split isn't pure
            if best['gain'] > 0:
                # Build a tree on the left
                left = self.__build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self.__build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        # Leaf node - value is the most common target value 
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        '''Build a decision tree classifier from the training set (X, y)'''
        # YOUR CODE HERE
        # Call a recursive function to build the tree
        self.root = self.__build(X, y)

    def predict_proba(self, X, tree):
        '''Predict class probabilities of the input samples X'''
        # YOUR CODE HERE
        # Leaf node
        if tree.value != None:
            return tree.value

        feature_value = X[tree.feature]
        
        # Go to the left
        if feature_value <= tree.threshold:
            return self.predict_proba(X=X, tree=tree.data_left)
        
        # Go to the right
        if feature_value > tree.threshold:
            return self.predict_proba(X=X, tree=tree.data_right)

    def predict(self, X):
      '''Predict class value for X'''
      # YOUR CODE HERE
      return [self.predict_proba(x, self.root) for x in X]